# 转换任意符合格式的数据到tmdat

In [ ]:
%load_ext autoreload

In [ ]:
!ls '/data/lyh/tianmoucData/tianmoucReconDataset_0701new/train/extreme/'

# 从tmdat读取一些数据做演示

- 你也可以用其他传感器或者仿真器读出来的数据

In [ ]:
%autoreload
import numpy as np
import matplotlib.pyplot as plt
from tianmoucv.data import TianmoucDataReader

data_path = '/data/lyh/tianmoucData/tianmoucReconDataset_0701new/train/extreme/HDR_train_tunnel1_hdr_blur'
dataset = TianmoucDataReader(data_path)

sample_0 = dataset[0]
sample_1 = dataset[1]

print('F0 Raw:',sample_0['F0_raw'].shape,sample_0['F0_raw'].type())
print('F1 Raw:',sample_0['F1_raw'].shape,sample_0['F0_raw'].type())
print('tsdiff Raw:',sample_1['rawDiff'].shape,sample_0['rawDiff'].type())

#打包10个完整周期
Len = 10
cone_list = []
rod_list = []

for i in range(Len):
    cone_list.append(dataset[i]['F0_raw'].numpy().astype(np.uint32))
for i in range(Len):  
    for t in range(dataset[i]['rawDiff'].shape[1]-1):
        rod_list.append(dataset[i]['rawDiff'][:,t,:,:].permute(1,2,0).numpy().astype(np.uint8))

# 调用转换接口转存到tmdat

In [ ]:
%autoreload
from collections import defaultdict
import numpy as np
import os
import struct
import cv2,sys
import os
from tianmoucv.rdp_usb import covert_to_tmdat

output_path = "/data/lyh/test_encode"
output_bandwidth_info = covert_to_tmdat(cone_list,rod_list,
                   output_path,
                   mode = 0, 
                   rod_adcprec = 8,
                   if_output=True)
'''
mode:0 并口 1串口 0慢一倍, 
    例: 757fps是0， 1515fps是1  1515@4bit是0
sample_list: 包含['cone']和['rod_list']的字典
        cone是uint32的320*320
        rod是uint8的 T 长度的list，每个rod为160*160*3，td，sdl，sdr布局
'''

# 以字节为单位的体积和带宽数据
print(output_bandwidth_info)

# 检查，对比转换前后的数据差异

In [ ]:
from tianmoucv.isp import vizDiff
import torch

aim = 4 # 读取的目标COP帧的帧号
N = 1   # read continue N frames （连续读取N帧COP以及对应的AOP）

dataset = TianmoucDataReader(data_path)
for index in range(aim,min(aim+1,len(dataset))):
    sample = dataset[index]
    tsdiff = torch.Tensor(sample['tsdiff'])
    length = tsdiff.shape[1]
    dataRatio = sample['dataRatio']
    print(sample['meta'])
    plt.figure(figsize=(12,6))  
    for i in range(N):
        F = sample['F'+str(i)].numpy()
        F_HDR = sample['F'+str(i)+'_HDR'].numpy()
        F_HDR[F_HDR>1]=1
        F_HDR[F_HDR<0]=0
        plt.subplot(N,6,1+i*6)
        plt.imshow(F)
        plt.subplot(N,6,2+i*6)
        plt.imshow(tsdiff[:,i*dataRatio,...].permute(1,2,0)*16)
        plt.subplot(N,6,3+i*6)
        plt.imshow(F_HDR)
        plt.show()
    break

dataset = TianmoucDataReader(output_path,strict=False)
for index in range(aim,min(aim+1,len(dataset))):
    sample = dataset[index]
    tsdiff = torch.Tensor(sample['tsdiff'])
    length = tsdiff.shape[1]
    dataRatio = sample['dataRatio']
    print(sample['meta'])
    plt.figure(figsize=(12,6))  
    for i in range(N):
        F = sample['F'+str(i)].numpy()
        F_HDR = sample['F'+str(i)+'_HDR'].numpy()
        F_HDR[F_HDR>1]=1
        F_HDR[F_HDR<0]=0
        plt.subplot(N,6,1+i*6)
        plt.imshow(F)
        plt.subplot(N,6,2+i*6)
        plt.imshow(tsdiff[:,i*dataRatio,...].permute(1,2,0)*16)
        plt.subplot(N,6,3+i*6)
        plt.imshow(F_HDR)
        plt.show()
    break